In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to='.'):
    """
    Unzips a file to the specified directory.

    :param zip_path: Path to the .zip file
    :param extract_to: Directory where files will be extracted (default: current directory)
    """
    if not zipfile.is_zipfile(zip_path):
        print("Error: Not a valid ZIP file.")
        return

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted '{zip_path}' to '{extract_to}'.")

# Example Usage
zip_path = "example.zip"  # Change this to your ZIP file path
extract_to = "output_folder"  # Change this to your desired extraction folder
unzip_file(zip_path, extract_to)


In [ ]:
!pip install numpy pandas matplotlib opencv-python tensorflow scikit-learn keras-tuner


In [ ]:
!pip install numpy pandas opencv-python tensorflow matplotlib seaborn scikit-learn keras


In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import itertools
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define paths
TRAIN_PATH = '/content/drive/MyDrive/data/train'
VALID_PATH = '/content/drive/MyDrive/data/valid'
TEST_PATH = '/content/drive/MyDrive/data/test'

# Image parameters
TARGET_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6

# Class names
class_names = {
    0: 'DangerousDriving',
    1: 'Distracted',
    2: 'Drinking',
    3: 'SafeDriving',
    4: 'SleepyDriving',
    5: 'Yawn'
}

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, TARGET_SIZE)
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

def prepare_dataset(annotations_df, base_path):
    images, labels = [], []
    for _, row in annotations_df.iterrows():
        image_path = os.path.join(base_path, row['image_name'])
        image = preprocess_image(image_path)
        images.append(image)
        labels.append(row['class_id'])
    return np.array(images), tf.keras.utils.to_categorical(labels, NUM_CLASSES)

# Data augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

def create_model(learning_rate=0.001, dropout_rate=0.5):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter tuning
model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'dropout_rate': [0.3, 0.5, 0.7],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

print(f"Best parameters: {grid_result.best_params_}")

# Train final model with best parameters
best_params = grid_result.best_params_
final_model = create_model(learning_rate=best_params['learning_rate'], dropout_rate=best_params['dropout_rate'])
final_model.fit(
    data_augmentation.flow(X_train, y_train, batch_size=best_params['batch_size']),
    epochs=best_params['epochs'],
    validation_data=(X_val, y_val),
    callbacks=[
        EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
        ModelCheckpoint('best_imagenet_model.keras', save_best_only=True, monitor='val_loss')
    ]
)

# Confusion Matrix & Classification Report
y_pred = final_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Classification Report:\n", classification_report(y_true, y_pred_classes))

# Plot confusion matrix
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(conf_matrix, list(class_names.values()))

def plot_predictions(model, test_images, test_labels, num_samples=10):
    indices = np.random.choice(len(test_images), num_samples, replace=False)
    sample_images = test_images[indices]
    sample_labels = test_labels[indices]
    predictions = model.predict(sample_images)

    plt.figure(figsize=(20, 20))
    for i in range(num_samples):
        plt.subplot(2, 5, i+1)
        plt.imshow(sample_images[i].astype('float32') / 255.0)
        actual = class_names[np.argmax(sample_labels[i])]
        predicted = class_names[np.argmax(predictions[i])]
        color = 'green' if actual == predicted else 'red'
        plt.title(f"Actual: {actual}\nPredicted: {predicted}", color=color)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display 10 test samples with predictions
plot_predictions(final_model, X_test, y_test)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt

# Define constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 5  # Adjust based on the dataset

def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freezing the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(hp.Int('units', min_value=64, max_value=512, step=64), activation='relu')(x)
    x = Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [1e-4, 1e-3, 1e-2])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Hyperparameter tuning
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='tuner_results',
    project_name='resnet50_tuning'
)

tuner.search(train_generator, validation_data=val_generator, epochs=10)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build and train the best model
best_model = build_model(best_hps)
best_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3), ModelCheckpoint('best_model.h5', save_best_only=True)]
)

# Fine-tuning
base_model = best_model.layers[0]
base_model.trainable = True
for layer in base_model.layers[:100]:  # Freezing first 100 layers
    layer.trainable = False

best_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

best_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3), ModelCheckpoint('fine_tuned_model.h5', save_best_only=True)]
)
